In [2]:
import pandas as pd
import pathlib

In [ ]:
caminho_bases = pathlib.Path('base de dados')
base_airbnb = pd.DataFrame()
meses = {'jan' : 1, 'fev' : 2, 'mar' : 3, 'abr' : 4, 'mai' : 5, 'jun' : 6, 'jul' : 7, 'ago' : 8, 'set' : 9, 'out': 10, 'nov' : 11, 'dez' : 12}


for arquivo in caminho_bases.iterdir():
    nome_mes = arquivo.name[:3]
    mes = meses[nome_mes]
    ano = arquivo.name[-8:]
    ano = int(ano.replace('.csv', ''))
    
    df = pd.read_csv(caminho_bases / arquivo.name)
    
    df['ano'] = ano
    df['mes'] = mes

    base_airbnb = base_airbnb._append(df)


display(base_airbnb)


- Temos muitas colunas, então vou analisar para usar apenas as colunas necessárias para a previsão dos preços.

- Para isso, vou criar um arquivo em excel com os 1000 primeiros registros e fazer uma analise qualitativa.

- Tipos de informações que eu vou excluir:
    -IDS, Links, Informações não relevantes
    -Colunas repetidadas ou extremamente parecidas com outras (que dão a mesma informação para o modelo)
    -Colunas preenchidas com texto livre (não irá ser rodado nenhuma análise de texto)
    -

In [19]:
print(list(base_airbnb.columns))
base_airbnb.head(1000).to_csv('primeiros registros.csv', sep= ';')

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [21]:
print(base_airbnb[['experiences_offered']].value_counts)

<bound method DataFrame.value_counts of       experiences_offered
0                    none
1                    none
2                    none
3                    none
4                    none
...                   ...
34324                none
34325                none
34326                none
34327                none
34328                none

[902210 rows x 1 columns]>
